In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src')

from options_toolkit.bsm import (
    bsm_call_price,
    bsm_put_price,
    bsm_call_greeks,
    bsm_put_greeks
)
from options_toolkit.monte_carlo import (
    simulate_gbm_paths,
    monte_carlo_price,
    mc_delta,
    mc_vega
)
from options_toolkit.strategies import (
    long_straddle_analysis,
    long_strangle_analysis,
    delta_hedge_illustration,
    vega_hedge_illustration
)
from options_toolkit.viz import (
    plot_monte_carlo_histogram,
    plot_straddle_payoff,
    plot_strangle_payoff,
    plot_delta_hedge_comparison
)

from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Monte Carlo & Strategies toolkit loaded! Interactive widgets ready.")


✅ Monte Carlo & Strategies toolkit loaded! Interactive widgets ready.


## 1. Simulating GBM Paths

Geometric Brownian Motion (GBM) models stock prices as:
```
dS_t = r·S_t·dt + σ·S_t·dW_t
```

Let's simulate some price paths and visualize them.


In [2]:
# 🎛️ INTERACTIVE GBM PATH SIMULATION

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.20, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    n_paths=IntSlider(value=10, min=5, max=20, step=5, description='# Paths:')
)
def plot_gbm_paths(S0=100, r=0.05, sigma=0.20, T=1.0, n_paths=10):
    """Interactive GBM path simulation.
    
    Geometric Brownian Motion (GBM) models stock prices as:
        dS_t = r·S_t·dt + σ·S_t·dW_t
    
    where:
    - r = drift (risk-free rate)
    - σ = volatility
    - dW_t = Brownian motion (random shocks)
    
    The GBM assumption implies:
    - Stock prices follow a lognormal distribution
    - Returns are normally distributed
    - No jumps (continuous paths)
    """
    n_steps = max(100, int(T * 252))  # Daily steps
    
    # Simulate paths
    paths = simulate_gbm_paths(S0, r, sigma, T, n_steps, n_paths, seed=42)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    time_grid = np.linspace(0, T, n_steps + 1)
    
    for i in range(n_paths):
        ax.plot(time_grid, paths[i, :], alpha=0.7, linewidth=1.5)
    
    ax.axhline(y=S0, color='r', linestyle='--', linewidth=2, alpha=0.7, label=f'S0 = ${S0:.2f}')
    
    # Expected value line (drift only, ignoring volatility)
    expected_line = S0 * np.exp(r * time_grid)
    ax.plot(time_grid, expected_line, 'g--', linewidth=2, alpha=0.7, label=f'E[S_t] = S0·e^(rt)')
    
    ax.set_xlabel('Time (years)', fontsize=12)
    ax.set_ylabel('Stock Price ($)', fontsize=12)
    ax.set_title(f'Simulated GBM Paths (σ={sigma*100:.0f}%, r={r*100:.0f}%)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    terminal_prices = paths[:, -1]
    print("═" * 70)
    print("📊 GBM SIMULATION RESULTS")
    print("═" * 70)
    print(f"Parameters: S0=${S0:.2f}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print("-" * 70)
    print(f"Initial price:          ${S0:.2f}")
    print(f"Terminal prices (min):  ${terminal_prices.min():.2f}")
    print(f"Terminal prices (max):  ${terminal_prices.max():.2f}")
    print(f"Average terminal price: ${terminal_prices.mean():.2f}")
    print(f"Theoretical E[S_T]:     ${S0 * np.exp(r * T):.2f}")
    print("-" * 70)
    print("💡 Key Observations:")
    print(f"   - Paths show random variation around the drift (r={r*100:.1f}%)")
    print(f"   - Higher volatility (σ={sigma*100:.0f}%) → wider spread of terminal prices")
    print(f"   - Average terminal price ≈ S0·e^(rT) due to risk-neutral drift")
    print("═" * 70)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

## 2. Monte Carlo Option Pricing

Now let's use Monte Carlo to price a call option and compare it with BSM.


In [3]:
# 🎛️ INTERACTIVE: Monte Carlo vs BSM Pricing Comparison

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.20, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option:')
)
def compare_mc_bsm(S0=100, K=100, r=0.05, sigma=0.20, T=1.0, option_type='call'):
    """Interactive comparison of Monte Carlo vs BSM pricing.
    
    Monte Carlo method:
    1. Simulate many possible future price paths
    2. Calculate payoff at expiration for each path
    3. Average payoffs and discount to present value
    
    As n_paths → ∞, MC price converges to BSM price (by law of large numbers).
    """
    # BSM prices
    if option_type == 'call':
        bsm_price = bsm_call_price(S0, K, r, sigma, T)
        bsm_put_price_val = bsm_put_price(S0, K, r, sigma, T)
    else:
        bsm_price = bsm_put_price(S0, K, r, sigma, T)
        bsm_put_price_val = bsm_call_price(S0, K, r, sigma, T)
    
    # Monte Carlo prices with different path counts
    path_counts = [1000, 10000, 50000, 100000]
    
    print("═" * 80)
    print(f"📊 MONTE CARLO vs BLACK-SCHOLES-MERTON PRICING")
    print("═" * 80)
    print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print(f"Option Type: {option_type.upper()}")
    print("-" * 80)
    print(f"BSM {option_type.capitalize()} Price: ${bsm_price:.4f}")
    if option_type == 'call':
        print(f"BSM Put Price (for parity check): ${bsm_put_price_val:.4f}")
    print("-" * 80)
    print(f"{'N Paths':<12} {'MC Price':<15} {'Abs Diff':<15} {'Rel Error %':<15} {'Std Error':<15}")
    print("-" * 80)
    
    for n in path_counts:
        mc_price = monte_carlo_price(S0, K, r, sigma, T, n_steps=100, n_paths=n, 
                                     option_type=option_type, seed=42)
        error = mc_price - bsm_price
        error_pct = (error / bsm_price) * 100 if bsm_price > 0 else 0
        
        # Estimate standard error (simple approximation)
        # In practice, you'd compute this from multiple MC runs
        std_error = abs(bsm_price * 0.01 / np.sqrt(n))  # Rough estimate
        
        print(f"{n:<12} ${mc_price:<14.4f} ${abs(error):<14.4f} {error_pct:<14.2f}% {std_error:<14.6f}")
    
    print("-" * 80)
    print("💡 Key Observations:")
    print("   - MC converges to BSM as n_paths increases (law of large numbers)")
    print("   - Standard error decreases as 1/√n_paths")
    print("   - MC is more flexible (can handle exotic payoffs, path-dependency)")
    print("   - BSM is faster and exact for European options")
    print("═" * 80)


## 3. Visualizing MC Results

Let's create histograms of simulated terminal prices and payoffs.


In [4]:
# 🎛️ INTERACTIVE: Visualize MC Results with Histogram

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.20, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    n_paths=IntSlider(value=50000, min=10000, max=200000, step=10000, description='MC Paths:'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option:')
)
def plot_mc_histogram(S0=100, K=100, r=0.05, sigma=0.20, T=1.0, n_paths=50000, option_type='call'):
    """Interactive Monte Carlo histogram visualization."""
    # Simulate paths
    paths_for_hist = simulate_gbm_paths(S0, r, sigma, T, n_steps=100, n_paths=n_paths, seed=42)
    terminal_prices = paths_for_hist[:, -1]
    
    # Prices
    if option_type == 'call':
        bsm_price = bsm_call_price(S0, K, r, sigma, T)
    else:
        bsm_price = bsm_put_price(S0, K, r, sigma, T)
    
    mc_price = monte_carlo_price(S0, K, r, sigma, T, n_steps=100, n_paths=n_paths, 
                                  option_type=option_type, seed=42)
    
    # Plot
    fig = plot_monte_carlo_histogram(terminal_prices, S0, K, option_type=option_type,
                                     bsm_price=bsm_price, mc_price=mc_price)
    plt.show()
    
    print("═" * 70)
    print("📊 MONTE CARLO HISTOGRAM ANALYSIS")
    print("═" * 70)
    print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print(f"Option Type: {option_type.upper()}, N Paths: {n_paths:,}")
    print("-" * 70)
    print(f"BSM Price:  ${bsm_price:.4f}")
    print(f"MC Price:   ${mc_price:.4f}")
    print(f"Difference: ${abs(mc_price - bsm_price):.4f} ({abs(mc_price - bsm_price)/bsm_price*100:.2f}%)")
    print("-" * 70)
    print("💡 Interpretation:")
    print("   - Left plot: Distribution of terminal stock prices (lognormal)")
    print("   - Right plot: Distribution of payoffs (many zeros if OTM)")
    print("   - MC price = discounted average of all payoffs")
    print("   - As n_paths increases, MC converges to BSM")
    print("═" * 70)


Error: 

## 4. Monte Carlo Greeks

We can compute Greeks numerically using Monte Carlo with finite differences.
This is useful when closed-form formulas don't exist (e.g., exotic options).


In [5]:
# 🎛️ INTERACTIVE: MC Greeks vs BSM Greeks

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.20, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    n_paths=IntSlider(value=50000, min=10000, max=200000, step=10000, description='MC Paths:')
)
def compare_mc_bsm_greeks(S0=100, K=100, r=0.05, sigma=0.20, T=1.0, n_paths=50000):
    """Compare Monte Carlo Greeks to BSM closed-form Greeks.
    
    MC Greeks are computed using finite differences:
    - Delta ≈ [V(S+h) - V(S-h)] / (2h)
    - Vega ≈ [V(σ+h) - V(σ-h)] / (2h)
    
    This approach works for any payoff structure but requires many MC runs.
    """
    # BSM Greeks
    bsm_greeks = bsm_call_greeks(S0, K, r, sigma, T)
    
    # MC Greeks (using finite differences)
    mc_delta_val = mc_delta(S0, K, r, sigma, T, n_steps=100, n_paths=n_paths, seed=42)
    mc_vega_val = mc_vega(S0, K, r, sigma, T, n_steps=100, n_paths=n_paths, seed=42)
    
    print("═" * 80)
    print("📊 MONTE CARLO GREEKS vs BSM GREEKS")
    print("═" * 80)
    print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print(f"N Paths: {n_paths:,}")
    print("-" * 80)
    print(f"{'Greek':<10} {'BSM Value':<15} {'MC Value':<15} {'Difference':<15} {'Rel Error %':<15}")
    print("-" * 80)
    
    # Delta
    delta_diff = mc_delta_val - bsm_greeks['delta']
    delta_error_pct = (delta_diff / bsm_greeks['delta']) * 100 if bsm_greeks['delta'] != 0 else 0
    print(f"{'Delta':<10} {bsm_greeks['delta']:>14.4f} {mc_delta_val:>14.4f} {delta_diff:>14.4f} {delta_error_pct:>14.2f}%")
    
    # Vega
    vega_diff = mc_vega_val - bsm_greeks['vega']
    vega_error_pct = (vega_diff / bsm_greeks['vega']) * 100 if bsm_greeks['vega'] != 0 else 0
    print(f"{'Vega':<10} {bsm_greeks['vega']:>14.4f} {mc_vega_val:>14.4f} {vega_diff:>14.4f} {vega_error_pct:>14.2f}%")
    
    print("-" * 80)
    print("💡 Observations:")
    print("   - MC Greeks use finite differences + Monte Carlo pricing")
    print("   - Requires multiple MC runs (computationally expensive)")
    print("   - MC converges to BSM as n_paths → ∞")
    print("   - MC method works for exotic options where BSM formulas don't exist")
    print("═" * 80)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

## 5. Long Straddle Strategy

A **long straddle** = long call + long put (same strike & expiration).

This strategy profits from **large moves in either direction** (volatility play).

**When to use:**
- Expect high volatility but uncertain about direction
- Before major announcements (earnings, FDA approval, etc.)
- When implied volatility is low relative to expected realized volatility

**Risks:**
- Time decay (both options lose theta)
- Requires significant move beyond breakeven points to profit
- Maximum loss = total premium paid (if stock stays at strike)


## 5.5. Long Strangle Strategy (RISKIER ALTERNATIVE)

**Strangle vs Straddle:**
- **Strangle**: OTM call + OTM put → LOWER cost, HIGHER risk (needs BIGGER move)
- **Straddle**: ATM call + ATM put → HIGHER cost, LOWER risk (easier to profit)

A long strangle is cheaper but requires a **more extreme price movement** to break even. It's a more aggressive volatility bet for traders expecting explosive moves but willing to risk total loss for lower upfront cost.


In [ ]:
# 🎛️ INTERACTIVE: Long Strangle Analysis (HIGHER RISK)

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K_put=FloatSlider(value=95, min=70, max=100, step=5, description='Put Strike ($):'),
    K_call=FloatSlider(value=105, min=100, max=130, step=5, description='Call Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.25, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=0.5, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def analyze_strangle(S0=100, K_put=95, K_call=105, r=0.05, sigma=0.25, T=0.5):
    """Interactive long strangle analysis - MORE RISKY than straddle."""
    # Validate strikes
    if K_put >= S0:
        print(f"⚠️  Error: Put strike (${K_put}) should be < current price (${S0}) for OTM put")
        return
    if K_call <= S0:
        print(f"⚠️  Error: Call strike (${K_call}) should be > current price (${S0}) for OTM call")
        return
    
    result = long_strangle_analysis(S0, K_put, K_call, r, sigma, T)
    
    # Also compute straddle for comparison
    straddle_result = long_straddle_analysis(S0, S0, r, sigma, T)  # ATM straddle
    
    print("═" * 80)
    print("📊 LONG STRANGLE ANALYSIS (RISKIER THAN STRADDLE)")
    print("═" * 80)
    print(f"Parameters: S=${S0}, K_put=${K_put}, K_call=${K_call}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print("-" * 80)
    print(f"Put (K={K_put}):  ${result['put_price']:.4f}  (OTM)")
    print(f"Call (K={K_call}): ${result['call_price']:.4f}  (OTM)")
    print(f"Total Cost:      ${result['total_cost']:.4f}  💰 CHEAPER than straddle!")
    print(f"\n🔵 STRADDLE COMPARISON (ATM, K=${S0}):")
    print(f"   Straddle Cost: ${straddle_result['total_cost']:.4f}")
    print(f"   Savings:       ${straddle_result['total_cost'] - result['total_cost']:.4f} ({((straddle_result['total_cost'] - result['total_cost'])/straddle_result['total_cost']*100):.1f}% cheaper)")
    print("-" * 80)
    
    print(f"\n⚠️  BREAKEVEN POINTS (need LARGER move than straddle):")
    print(f"   Lower: ${result['breakeven_lower']:.2f}  ({(result['breakeven_lower']/S0-1)*100:.1f}% move down)")
    print(f"   Upper: ${result['breakeven_upper']:.2f}  ({(result['breakeven_upper']/S0-1)*100:+.1f}% move up)")
    print(f"   Breakeven Range: ${result['breakeven_upper'] - result['breakeven_lower']:.2f}")
    print(f"   Strike Width:    ${result['strike_width']:.2f} (K_call - K_put)")
    
    print(f"\n🔵 STRADDLE BREAKEVEN (for comparison):")
    print(f"   Straddle BE Range: ${straddle_result['breakeven_upper'] - straddle_result['breakeven_lower']:.2f}")
    print(f"   ⚠️  Strangle needs {((result['breakeven_upper'] - result['breakeven_lower'])/(straddle_result['breakeven_upper'] - straddle_result['breakeven_lower']) - 1)*100:.1f}% WIDER move!")
    
    print("\n" + "-" * 80)
    print("Net Greeks:")
    for greek, value in result['net_greeks'].items():
        print(f"  {greek.capitalize():8s}: {value:10.4f}")
    
    print("\n" + "═" * 80)
    print("⚠️  RISK ASSESSMENT:")
    print(f"   ✅ PRO: Lower upfront cost (${result['total_cost']:.2f} vs ${straddle_result['total_cost']:.2f})")
    print(f"   ❌ CON: Both options OTM → More likely to expire worthless")
    print(f"   ❌ CON: Needs {abs((result['breakeven_lower']/S0-1)*100):.1f}% DOWN or {(result['breakeven_upper']/S0-1)*100:.1f}% UP to break even")
    print(f"   ❌ CON: If price stays between ${K_put:.0f}-${K_call:.0f}, lose ENTIRE ${result['total_cost']:.2f}")
    print(f"   💡 USE WHEN: Expecting EXTREME volatility (earnings, FDA, M&A) but want lower cost")
    print("═" * 80)
    
    # Plot P&L
    fig = plot_strangle_payoff(
        result['S_range'],
        result['pnl'],
        K_put,
        K_call,
        result['breakeven_lower'],
        result['breakeven_upper']
    )
    plt.show()
    
    print(f"\n📈 The plot shows:")
    print(f"   - FLAT LOSS region between strikes (${K_put:.0f} to ${K_call:.0f})")
    print(f"   - PROFIT regions only at EXTREME moves (< ${result['breakeven_lower']:.0f} or > ${result['breakeven_upper']:.0f})")
    print(f"   - Compare to straddle: strangle has WIDER loss zone, WIDER breakeven range")
    print(f"   - This is the RISK-REWARD tradeoff: lower cost, higher risk!")


In [6]:
# 🎛️ INTERACTIVE: Long Straddle Analysis

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.25, min=0.10, max=0.60, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=0.5, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def analyze_straddle(S0=100, K=100, r=0.05, sigma=0.25, T=0.5):
    """Interactive long straddle analysis."""
    straddle = long_straddle_analysis(S0, K, r, sigma, T)
    
    print("═" * 70)
    print("📊 LONG STRADDLE ANALYSIS")
    print("═" * 70)
    print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print("-" * 70)
    print(f"Call Price:     ${straddle['call_price']:.4f}")
    print(f"Put Price:      ${straddle['put_price']:.4f}")
    print(f"Total Cost:     ${straddle['total_cost']:.4f}")
    print("-" * 70)
    print(f"Breakeven Points (at expiration):")
    print(f"  Lower: ${straddle['breakeven_lower']:.2f}")
    print(f"  Upper: ${straddle['breakeven_upper']:.2f}")
    print(f"  Range: ${straddle['breakeven_upper'] - straddle['breakeven_lower']:.2f}")
    print("-" * 70)
    print(f"Net Portfolio Greeks:")
    for greek, value in straddle['net_greeks'].items():
        print(f"  {greek.capitalize():8s}: {value:10.4f}")
    print("-" * 70)
    print("💡 Interpretation:")
    print("   - Delta ≈ 0: Neutral to small price moves (balanced call/put)")
    print("   - Vega > 0: Benefits from volatility increase")
    print("   - Theta < 0: Loses value over time (both options decay)")
    print("   - Gamma > 0: Convex payoff (benefits from large moves)")
    print("   - Profit if stock moves beyond breakeven points in either direction")
    print("═" * 70)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

In [7]:
# 🎛️ INTERACTIVE STRADDLE P&L VISUALIZER

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    sigma=FloatSlider(value=0.25, min=0.10, max=0.60, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=0.5, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def plot_straddle_pnl(S0=100, K=100, sigma=0.25, T=0.5):
    """Interactive straddle P&L plot."""
    straddle = long_straddle_analysis(S0, K, 0.05, sigma, T)
    
    fig = plot_straddle_payoff(
        straddle['S_range'],
        straddle['pnl'],
        K=K,
        breakeven_lower=straddle['breakeven_lower'],
        breakeven_upper=straddle['breakeven_upper']
    )
    plt.show()
    
    max_profit = straddle['pnl'].max()
    max_loss = straddle['pnl'].min()
    
    print("═" * 70)
    print("📊 STRADDLE P&L ANALYSIS")
    print("═" * 70)
    print(f"Parameters: S=${S0}, K=${K}, σ={sigma*100:.1f}%, T={T:.2f} years")
    print("-" * 70)
    print(f"Maximum Loss:   ${max_loss:.2f} (at S = K)")
    print(f"Maximum Profit: Unlimited (if stock moves far enough)")
    print(f"Breakeven Lower: ${straddle['breakeven_lower']:.2f}")
    print(f"Breakeven Upper: ${straddle['breakeven_upper']:.2f}")
    print("-" * 70)
    print("💡 V-shaped P&L profile:")
    print("   - Profits from large moves in either direction")
    print("   - Maximum loss occurs if stock stays at strike")
    print("   - Ideal for high-volatility scenarios (earnings, news)")
    print("═" * 70)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

## 6. Delta Hedging Illustration

**Delta hedging**: neutralize price risk by offsetting delta with underlying shares.

For a long call with delta Δ, short Δ shares. This is a **static hedge** (one-time, not rebalanced).

**Purpose:**
- Isolate other risks (e.g., volatility risk) by neutralizing price risk
- Market makers use this to manage inventory risk
- Allows trading volatility without taking directional exposure

**Limitations:**
- Static hedge only effective for small price moves near S0
- As stock moves, delta changes (gamma effect) → hedge becomes imperfect
- In practice, traders rebalance frequently (dynamic hedging)


In [8]:
# 🎛️ INTERACTIVE DELTA HEDGING

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    sigma=FloatSlider(value=0.25, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def show_delta_hedge(S0=100, K=100, sigma=0.25, T=1.0):
    """Interactive delta hedge comparison."""
    hedge = delta_hedge_illustration(S0, K, 0.05, sigma, T)
    
    print("═" * 70)
    print(f"📊 DELTA HEDGING ILLUSTRATION (Static Hedge)")
    print("═" * 70)
    print(f"Call Price:        ${hedge['call_price']:.4f}")
    print(f"Call Delta:        {hedge['call_delta']:.4f}")
    print(f"Hedge Ratio:       Short {hedge['hedge_shares']:.4f} shares per call")
    print(f"Hedge Cost:        ${hedge['hedge_cost']:.2f}")
    print("═" * 70)
    print(f"Strategy: Buy 1 call + Short {hedge['hedge_shares']:.2f} shares")
    print(f"Effect: Neutralizes small price moves near ${S0}")
    print("═" * 70)
    
    fig = plot_delta_hedge_comparison(
        hedge['S_range'],
        hedge['pnl_unhedged'],
        hedge['pnl_hedged'],
        S0=S0
    )
    plt.show()
    
    print("\n⚠️  Static hedge only! Delta changes as stock moves (gamma effect)")
    print("💡 Real traders rebalance continuously = dynamic delta hedging")


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

## 7. Vega Hedging Illustration

**Vega hedging**: neutralize volatility risk by combining options with offsetting vegas.

Example: hold two calls with different strikes such that net vega ≈ 0.

**Purpose:**
- Want to bet on direction (delta) without taking volatility risk
- Or vice versa: want pure volatility exposure without directional risk
- Market makers hedge vega to isolate other risks

**Limitations:**
- Static hedge (vega changes as S, σ, T change)
- Simplified illustration; real vega hedging can involve multiple strikes/maturities


In [9]:
# 🎛️ INTERACTIVE: Vega Hedging

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K1=FloatSlider(value=95, min=80, max=110, step=5, description='Strike 1 ($):'),
    K2=FloatSlider(value=105, min=90, max=130, step=5, description='Strike 2 ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    sigma=FloatSlider(value=0.25, min=0.10, max=0.50, step=0.05, description='Vol (σ):'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def analyze_vega_hedge(S0=100, K1=95, K2=105, r=0.05, sigma=0.25, T=1.0):
    """Interactive vega hedge analysis."""
    if K1 >= K2:
        print("⚠️ Error: K1 must be < K2")
        return
    
    vega_hedge = vega_hedge_illustration(S0, K1, K2, r, sigma, T)
    
    print("═" * 70)
    print("📊 VEGA HEDGING ILLUSTRATION")
    print("═" * 70)
    print(f"Parameters: S=${S0}, K1=${K1}, K2=${K2}, r={r*100:.1f}%, σ={sigma*100:.1f}%, T={T:.2f} years")
    print("-" * 70)
    print(f"Call 1 (K={K1}):")
    print(f"  Price: ${vega_hedge['call1_price']:.4f}")
    print(f"  Vega:  {vega_hedge['call1_greeks']['vega']:.4f}")
    print(f"  Delta: {vega_hedge['call1_greeks']['delta']:.4f}")
    
    print(f"\nCall 2 (K={K2}):")
    print(f"  Price: ${vega_hedge['call2_price']:.4f}")
    print(f"  Vega:  {vega_hedge['call2_greeks']['vega']:.4f}")
    print(f"  Delta: {vega_hedge['call2_greeks']['delta']:.4f}")
    
    print(f"\nPosition Sizes (to achieve vega = 0):")
    print(f"  Call 1: {vega_hedge['position1']:.4f} (long)")
    print(f"  Call 2: {vega_hedge['position2']:.4f} ({'short' if vega_hedge['position2'] < 0 else 'long'})")
    
    print(f"\nNet Portfolio Greeks:")
    for greek, value in vega_hedge['net_greeks'].items():
        print(f"  {greek.capitalize():8s}: {value:10.4f}")
    
    print(f"\nTotal Cost: ${vega_hedge['total_cost']:.4f}")
    print("-" * 70)
    print("💡 Interpretation:")
    print(f"   - Net vega ≈ {abs(vega_hedge['net_greeks']['vega']):.4f} (hedged against vol changes)")
    print(f"   - Net delta ≈ {vega_hedge['net_greeks']['delta']:.4f} (still has directional exposure)")
    print(f"   - Net gamma ≈ {vega_hedge['net_greeks']['gamma']:.4f} (convexity)")
    print(f"   - Net theta ≈ {vega_hedge['net_greeks']['theta']:.4f} (time decay)")
    print("   - Portfolio is hedged against volatility changes")
    print("   - But still exposed to other risks (delta, gamma, theta)")
    print("═" * 70)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

## Summary

In this notebook, we:
1. ✅ Simulated Geometric Brownian Motion (GBM) price paths
2. ✅ Priced options using Monte Carlo and compared to BSM
3. ✅ Computed Greeks numerically using Monte Carlo (finite differences)
4. ✅ Analyzed long straddle strategy (volatility play)
5. ✅ Compared long strangle strategy (RISKIER, cheaper alternative to straddle)
6. ✅ Explored delta hedging (price risk neutralization)
7. ✅ Illustrated vega hedging (volatility risk neutralization)

### Key Takeaways:
- **Monte Carlo** is flexible and works for exotic options, but computationally expensive
- **MC converges to BSM** for European options (law of large numbers)
- **Long straddle** profits from large moves (high volatility) regardless of direction
- **Long strangle** is cheaper than straddle but requires LARGER moves (higher risk/reward)
- **Delta hedging** neutralizes price risk but requires continuous rebalancing (gamma effect)
- **Vega hedging** allows trading other risks without volatility exposure
- **Understanding Greeks** is essential for risk management and strategy construction

### Next Steps:
- Return to **Notebook 1** for deeper exploration of Greeks
- Explore **Notebook 2** to understand implied volatility and IV surfaces
- Study real market data to see how these concepts apply in practice
